# Fixed Point Precision Study on LSTM-250

In [1]:
import torch
import numpy as np

# Load the Floating-Point Model and Dataset

## Load Saved Model Weights

In [7]:
!ls -ltrh ./session/
print('')

# Load saved model dictionary
model_path = './session/model-params-np.pt'
model_dict = torch.load(model_path)
print(model_dict.keys())

total 1.7G
-rw-rw-r-- 1 makabir makabir  310 Jun 24 16:57 README.md
-rw-rw-r-- 1 makabir makabir 1.3M Jun 26 12:52 curated-dataset.pt
-rw-rw-r-- 1 makabir makabir 5.4M Jun 27 18:24 trained-lstm250-83.09p.pt
-rw-rw-r-- 1 makabir makabir  829 Jun 27 18:30 label-to-index.pt
-rw-rw-r-- 1 makabir makabir 167M Jun 28 23:12 test-features.pt
-rw-rw-r-- 1 makabir makabir 1.2G Jun 28 23:15 train-features.pt
-rw-rw-r-- 1 makabir makabir 125M Jun 28 23:48 Test_feat.pt
-rw-rw-r-- 1 makabir makabir 5.4M Jun 28 23:48 trained-lstm250-82.26p.pt
-rw-rw-r-- 1 makabir makabir 125M Jun 28 23:49 Test_feat-82.26p.pt
drwxrwxr-x 2 makabir makabir 4.0K Jun 29 13:23 backup
-rw-rw-r-- 1 makabir makabir 5.4M Jun 29 15:14 trained-lstm250.pt
-rw-rw-r-- 1 makabir makabir 125M Jun 29 15:28 test-export-ds.pt
-rw-rw-r-- 1 makabir makabir 7.9M Jun 29 18:06 model-params-np.pt

dict_keys(['accuracy', 'correct_count', 'Hparam', 'index_to_label', 'weights_dict'])


In [8]:
# Parse the values for easier use
Accuracy = model_dict['accuracy']
Correct_count = model_dict['correct_count']
Index_to_label = model_dict['index_to_label']
Label_to_index = {label:index for index, label in Index_to_label.items()}
Hparam = model_dict['Hparam']
Weights_np = model_dict['weights_dict']
Model_perf = f'Model Performance:   accuracy: {Accuracy:.2f}%   correct_count: {Correct_count}'  # to be used later
print('Hparam:', Hparam)
print('Model_perf:', Model_perf)


Hparam: {'input_size': 123, 'hidden_size': 250, 'num_layers': 3, 'num_classes': 39}
Model_perf: Model Performance:   accuracy: 73.03%   correct_count: 5355


## Load Dataset

In [9]:
# Prints information about dataset item
def print_dataItem(item):
    mstr = f"label: {item[0]}, label_index: {item[1]}, predicted_index: {item[2]}, sequence_length: {item[3]},"
    mstr2 = f"\nfeature_sequence shape: {item[4].shape}, feature_seq type: {type(item[4])}"
    print(mstr, mstr2)

    
# Load the test dataset
ds_path = "./session/test-export-ds.pt"
DS_loaded = torch.load(ds_path)
print(DS_loaded.keys())
print('DS_loaded len:', len(DS_loaded['dataset']))
print('schema:', DS_loaded['dataset_schema'])


# Make sure the label-to-index dictionary matches the one in the model
for key in DS_loaded['label_dict']:
    assert DS_loaded['label_dict'][key] == Label_to_index[key], 'Dataset and Model Label-to-index are different'
print('INFO: Dataset and Model label-to-index matched')


# show an item information
DataItems = DS_loaded['dataset']
item = DataItems[0]
print_dataItem(item)

dict_keys(['label_dict', 'dataset_schema', 'dataset'])
DS_loaded len: 7333
schema: (label, label_index, predicted_index, sequence_length, feature_sequence)
INFO: Dataset and Model label-to-index matched
label: h#, label_index: 6, predicted_index: 6, sequence_length: 38, 
feature_sequence shape: (38, 123), feature_seq type: <class 'numpy.ndarray'>


In [10]:
# Delete names to avoid confusions later
del model_path, model_dict
del ds_path, key, item

# Define Fixed-Point Methods

In [13]:
# Delete the cache before importing
!rm -rf __pycache__/
from AK_FixedPoint import *

# Run Unit tests to make sure everything is okay
!python3 unittest_fxp.py

fxp_ctor() tests:
[8, 9, 4, 2, 31, -3, -9]
[-8. -7.  4.  2. -1. -3.  7.]
fxp_Status(overflow=True, overflow_count=4, max_vali=496, min_vali=-144, max_valf=31.0, min_valf=-9.0)

fxp_Status(overflow=True, overflow_count=4, max_vali=1024, min_vali=-1024, max_valf=32.0, min_valf=-32.0)
[16, 18, -4, -32, 32, -10]
[-16. -14.  -4.   0.   0. -10.]


fxp_getSignBits() tests:
fxp_Status(overflow=False, overflow_count=0, max_vali=112, min_vali=-80, max_valf=7.0, min_valf=-5.0)
[1, 1, 1, 0, 0, 0]
[1 1 1 0 0 0]

fxp_Status(overflow=False, overflow_count=0, max_vali=448, min_vali=-320, max_valf=14.0, min_valf=-10.0)
[1, 0, 1, 0, 1, 0]
[1 0 1 0 1 0]


fxp_computeStatus tests:
fxp_Status(overflow=True, overflow_count=4, max_vali=496, min_vali=-144, max_valf=31.0, min_valf=-9.0)
[ 128  144   64   32  496  -48 -144]
[-128 -112   64   32  -16  -48  112]
fxp_Status(overflow=True, overflow_count=4, max_vali=496, min_vali=-144, max_valf=31.0, min_valf=-9.0)

fxp_Status(overflow=True, overflow_count=5, max_v

# Check Fixed-Point Precisions of Dataset and Weights

## Dataset

In [36]:
from math import inf as INF


all_features = []
for item in DataItems:
    all_features.append(item[-1])
all_features = np.concatenate(all_features)
all_features = all_features.flatten()
all_feat_0 = all_features[all_features!=0]     # remove all zeros

print('all_features:', all_features.shape)
print('all_feat_0  :', all_feat_0.shape)

all_features: (21014796,)
all_feat_0  : (21001647,)


In [37]:
print('min_ds    :', np.min(all_features))
print('max_ds    :', np.max(all_features))
print('min_abs_ds:', np.min(np.abs(all_features)))
print('max_abs_ds:', np.max(np.abs(all_features)))
print('min_abs_0 :', np.min(np.abs(all_feat_0)))

min_ds    : -100.0
max_ds    : 9.921182
min_abs_ds: 0.0
max_abs_ds: 100.0
min_abs_0 : 2.506533e-16


## Weights

In [42]:
all_weights = []
for k, weight in Weights_np.items():
    all_weights.append(weight.flatten())
all_weights = np.concatenate(all_weights)
all_weights = all_weights.flatten()
all_w_0 = all_weights[all_weights!=0]    # remove all zeros

print('all_weights:', all_weights.shape)
print('all_w_0    :', all_w_0.shape)

all_weights: (1388789,)
all_w_0    : (1388789,)


In [41]:
print('min_w    :', np.min(all_weights))
print('max_w    :', np.max(all_weights))
print('min_abs_w:', np.min(np.abs(all_weights)))
print('max_abs_w:', np.max(np.abs(all_weights)))
print('min_abs_0:', np.min(np.abs(all_w_0)))

min_w    : -0.1854084
max_w    : 0.21388349
min_abs_w: 3.3071996e-13
max_abs_w: 0.21388349
min_abs_0: 3.3071996e-13
